In [11]:
%load_ext autoreload
%autoreload 2
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import logging
import numpy as np
import matplotlib.pyplot as plt
from cerebra_atlas_python.cerebra import CerebrA 
from cerebra_atlas_python.plotting import get_cmap_colors,add_grid_to_ax, remove_ax
from cerebra_atlas_python.utils import setup_logging

In [13]:
setup_logging(level=logging.INFO)

In [14]:
cerebra = CerebrA()
cerebra.label_details.to_csv("text.csv")
cerebra.label_details

 [INFO] 2023-12-05 16:25:15.55 mni_average - __init__: Using data from SUBJECTS_DIR/MNIAverage
 [INFO] 2023-12-05 16:25:15.55 mni_average - _set_bem: Loading boundary element model from disk | MNIAverage_bem_0.33_0.0042_0.33_ico_4
 [INFO] 2023-12-05 16:25:15.135 mni_average - _set_vol_src: Reading volume source space from disk


,Mindboggle ID,Label Name,CerebrA ID,hemisphere,cortical,color
0,0.0,Empty,0,,False,#000000
1,2027.0,Rostral Middle Frontal,1,Right,True,#ff001e
2,631.0,Vermal lobules VI-VII,2,Right,False,#ff0013
3,2009.0,Inferior temporal,3,Right,True,#ff0003
4,58.0,Accumbens Area,4,Right,False,#ff0700
...,...,...,...,...,...,...
99,53.0,Hippocampus,99,Left,False,#ff00f0
100,50.0,Caudate,100,Left,False,#ff00e5
101,630.0,Vermal lobules I-V,101,Left,False,#ff00d4
102,1031.0,Supramarginal,102,Left,True,#ff00ca


In [15]:
%matplotlib qt

In [17]:
# CORTICAL vs NON-CORTICAL pie chart
n_cortical = np.sum(cerebra.label_details["cortical"])
n_non_cortical = 102 - n_cortical
plt.pie([n_cortical, n_non_cortical],labels=[f"Cortical (N={n_cortical})", f"Non-cortical (N={n_non_cortical})"], colors=[cerebra.cortical_color,cerebra.non_cortical_color])

([<matplotlib.patches.Wedge at 0x7f4e397d2b90>,
 [Text(-0.3655903556118915, 1.0374698510721025, 'Cortical (N=62)'),
  Text(0.3655904527468272, -1.037469816843059, 'Non-cortical (N=40)')])

In [18]:
cerebra.plot_3d(plot_regions=True, plot_src_space=False, density=4, plot_cortical=True)

(<Figure size 640x480 with 1 Axes>,
 <Axes3D: xlabel='X (R)', ylabel='Y (A)', zlabel='Z (S)'>)

In [19]:
cerebra.volume_data_sparse.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103])

In [21]:
plt.close("all")

PLOT_ONLY_CORTICAL = False 
PLOT_ONLY_NON_CORTICAL = False 
# Else plot both
INCLUDE_WHITE_MATTER = True 


# LOG scale for histogram
# Distribution of points per region
# Keep only leftside labels (symmetrical)
leftside_label_details = cerebra.label_details[cerebra.label_details['hemisphere'] == "Left"].copy()
wm_label_details = cerebra.label_details[cerebra.label_details['Label Name'] == "White matter"].copy()

n_points = np.array([cerebra.get_n_points_from_region_id(region_id) for region_id in leftside_label_details['CerebrA ID'].values])
leftside_label_details["npoints"] = n_points

cortical_label_details = leftside_label_details.loc[leftside_label_details['cortical'] == True,:].copy()
non_cortical_label_details = leftside_label_details.loc[leftside_label_details['cortical'] == False,:].copy()

leftside_label_details.sort_values(by=['npoints'],inplace=True)
cortical_label_details.sort_values(by=['npoints'],inplace=True)
non_cortical_label_details.sort_values(by=['npoints'],inplace=True)

white_matter_n_points = cerebra.get_n_points_from_region_id(103)
empty_n_points = cerebra.get_n_points_from_region_id(0)



# Define data and colors for each bar
if PLOT_ONLY_CORTICAL:
    data = np.array([*cortical_label_details["npoints"].values])
    colors = np.array([*cortical_label_details["color"].values])
    labels = np.array([*cortical_label_details["Label Name"].values])
    label_ids = np.array(cortical_label_details['CerebrA ID'].values)
elif PLOT_ONLY_NON_CORTICAL:
    data = np.array([*non_cortical_label_details["npoints"].values])
    colors = np.array([*non_cortical_label_details["color"].values])
    labels = np.array([*non_cortical_label_details["Label Name"].values])
    label_ids = np.array(non_cortical_label_details['CerebrA ID'].values)
elif INCLUDE_WHITE_MATTER:
    data = np.array([white_matter_n_points,*leftside_label_details["npoints"].values])
    colors = np.array([wm_label_details["color"].item(), *leftside_label_details["color"].values])
    labels = np.array([wm_label_details["Label Name"].item(),*leftside_label_details["Label Name"].values])
    label_ids = np.array([wm_label_details["CerebrA ID"].item(),*leftside_label_details['CerebrA ID'].values])
else:
    data = np.array([*leftside_label_details["npoints"].values])
    colors = np.array([ *leftside_label_details["color"].values])
    labels = np.array([*leftside_label_details["Label Name"].values])
    label_ids = np.array([*leftside_label_details['CerebrA ID'].values])
    
# PIE CHART
fig2,ax2 = plt.subplots()
fig2.set_size_inches(12, 10)
threshold = data.sum() * 0.01
labels_pie = [l if data[i] > threshold else "" for i,l in enumerate(labels)]
ax2.pie(data,colors=colors,labels=labels_pie)
plt.show()
# HISTOGRAM
fig, ax = plt.subplots()
ax = remove_ax(ax,keep_names=["left","top"])    
    
for i, n_pts in enumerate(data):
    if n_pts == 0:
        continue
    p =ax.barh(i,n_pts, color=colors[i], linestyle='-', height=1,label=labels[i],edgecolor = "white", linewidth=0)
    # ax.bar_label(p, label_type='edge', padding=10,fontsize=16)
    ax.bar_label(p, label_type='edge', padding=10,fontsize=16,fontweight='bold')

add_grid_to_ax(ax)
ax.xaxis.tick_top()

fig.suptitle(f"Distribution of points per {'(cortical)' if PLOT_ONLY_CORTICAL else '(non-cortical)' if PLOT_ONLY_NON_CORTICAL else ''} region")

ax.set_xlabel("Number of points")
ax.set_xticks((np.linspace(0,data.max(),num=20) / 100).astype(int) *100)

# # Add labels to bins

for i,(name,id) in enumerate(zip(labels,label_ids)):
    ax.text(-data.max()*0.02,i, f"{name} id:{id}",fontsize=16,verticalalignment='center',horizontalalignment='right')

fig.set_size_inches(28.5, 15.5)




mngr = plt.get_current_fig_manager()
geom = mngr.window.geometry()
x,y,dx,dy = geom.getRect()
mngr.window.setGeometry(2600, 00, dx, dy)

# mngr.window.setGeometry(500,100,640, 545)

In [22]:
# Plot cortical region distribution
print(f"Source space n points: {cerebra.get_src_space_pc().shape}")
# Visualize source spaceremove_axis
cerebra.plot_3d(plot_regions=False, plot_src_space=True)
cerebra.orthoview(plot_regions=False,plot_src_space=True,s=1)

Source space n points: (16426, 3)


(<Figure size 1500x1500 with 4 Axes>,
 array([[<Axes: xlabel='Y', ylabel='Z'>, <Axes: xlabel='X', ylabel='Z'>],
        [<Axes: xlabel='X', ylabel='Y'>, <Axes: >]], dtype=object))

In [26]:
cerebra.get_src_space_pc().shape

(16426, 3)

In [48]:
plt.close("all")

region_ids = np.array([cerebra.get_region_id_from_point(pt) for pt in cerebra.get_src_space_pc()])
# plt.hist(n_points)
cerebra.label_details["vol_src_count"] = np.bincount(region_ids)
data = np.array([*cerebra.label_details["vol_src_count"].values])
colors = np.array([*cerebra.label_details["color"].values])
labels = np.array([*cerebra.label_details["Label Name"].values])
label_ids = np.array([*cerebra.label_details['CerebrA ID'].values])

# HISTOGRAM
fig, ax = plt.subplots()
ax = remove_ax(ax,keep_names=["left","top"])    
    
for i, n_pts in enumerate(data):
    if n_pts == 0:
        continue
    p =ax.barh(i,n_pts, color=colors[i], linestyle='-', height=1,label=labels[i],edgecolor = "white", linewidth=0)
    # ax.bar_label(p, label_type='edge', padding=10,fontsize=16)
    ax.bar_label(p, label_type='edge', padding=10,fontsize=12,fontweight='bold')

add_grid_to_ax(ax)
ax.xaxis.tick_top()

fig.suptitle(f"Distribution of regions in the source space")

ax.set_xlabel("Number of points in source space")
ax.set_xticks((np.linspace(0,data.max(),num=20) / 100).astype(int) *100)

# # Add labels to bins

for i,(name,id) in enumerate(zip(labels,label_ids)):
    ax.text(-data.max()*0.02,i, f"{name} id:{id}",fontsize=12,verticalalignment='center',horizontalalignment='right')

fig.set_size_inches(28.5, 15.5)




mngr = plt.get_current_fig_manager()
geom = mngr.window.geometry()
x,y,dx,dy = geom.getRect()
mngr.window.setGeometry(2600, 00, dx, dy)
